# Simulator

In [1]:
import pandas as pd

import payment_simulator as ps
from payment_simulator.anomaly import AnomalyGenerator
from payment_simulator.networks import SimplePaymentNetwork
from payment_simulator.utils import random_payment_timing, random_payment_value

In [2]:
TOTAL_BANKS = 20
INITIAL_BANKS = 3
INCREMENT_BANKS = 5
ALPHA = 0.00001
AVG_PAYMENTS = 35
ALLOW_SELF_LOOP = False

In [3]:
sim_periods = list(range(15))

sim_params = {
    "open_time": "06:30:00",
    "close_time": "18:30:00",
    "value_fn": random_payment_value,
    "timing_fn": random_payment_timing,
}

payment_network = SimplePaymentNetwork(
    total_banks=TOTAL_BANKS,
    avg_payments=AVG_PAYMENTS,
    alpha=ALPHA,
    allow_self_loop=ALLOW_SELF_LOOP
)

anomaly_generator = AnomalyGenerator(
    anomaly_start=5,
    anomaly_end=12,
    prob_start=0.8,
    prob_end=1,
    lambda_start=0.5,
    lambda_end=2.5,
    rate=0.5,
)

In [4]:
normal_transactions = ps.TransactionSim(sim_id=1, network=payment_network, **sim_params)
normal_transactions.run(sim_periods)

payments1 = normal_transactions.get_payments_df()

pd.concat([payments1.head(3), payments1.tail(3)])

,Period,Time,Sender,Receiver,Count,Value
0,0,13:18:55,0,4,1,0.768472
1,0,15:02:34,0,4,1,0.267236
2,0,07:57:41,0,4,1,0.260640
10497,14,07:54:40,18,3,1,2.021458
10498,14,18:29:31,18,7,1,2.935881
10499,14,09:46:06,18,2,1,0.301337


In [5]:
anomaly_transactions = ps.AnomalyTransactionSim(sim_id=2, network=payment_network, anomaly=anomaly_generator, **sim_params)
anomaly_transactions.run(sim_periods)

payments2 = anomaly_transactions.get_payments_df()

pd.concat([payments2.head(3), payments2.tail(3)])

,Period,Time,Sender,Receiver,Count,Value
0,0,09:57:06,0,7,1,1.337997
1,0,16:25:05,0,7,1,5.006299
2,0,10:35:43,0,7,1,3.710578
10497,14,14:03:33,19,16,1,0.850998
10498,14,14:13:30,19,16,1,0.980043
10499,14,13:54:27,19,5,1,0.287569


In [13]:
print(f"Total Value of Normal RTGS  : {payments1['Value'].sum():.3f} from {payments1.shape[0]} transactions")
print(f"Total Value of Anomaly RTGS : {payments2['Value'].sum():.3f} from {payments1.shape[0]} transactions")

Total Value of Normal RTGS  : 17092.819 from 10500 transactions
Total Value of Anomaly RTGS : 26931.659 from 10500 transactions


In [7]:
correct = 0
test_len = 5

for _ in range(test_len):
    normal_transactions.run(sim_periods)
    anomaly_transactions.run(sim_periods)

    normal_df = normal_transactions.get_payments_df()
    normal_df = normal_df[(normal_df["Period"] <=12) & (normal_df["Period"] >= 5)]

    anomaly_df = anomaly_transactions.get_payments_df()
    anomaly_df = anomaly_df[(anomaly_df["Period"] <=12) & (normal_df["Period"] >= 5)]

    x1 = normal_df["Value"].sum()
    x2 = anomaly_df["Value"].sum()

    if x2 > x1:
        correct += 1

print(f"Success rate: {correct / test_len * 100:.2f}%")

Success rate: 100.00%
